In [42]:
from path_to_folders import images_path, labels_path, masks_path
import tensorflow as tf
import glob

### Define variables

In [43]:
CLASSES = 2
COLOR_MAP = {
    0: 'magenta',
    1: 'yellow'
}

SAMPLE_SIZE = (384, 384)
OUTPUT_SIZE = (768, 768)

### Function imege load

In [49]:
def image_load(image, mask):
    # Load, decoder and resize image .jpg
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize(image, OUTPUT_SIZE)

    # Convert image data type to float32 and normalizes the pixels to the range [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image/ 255.0

    # Load, decoder and resize mask .png
    mask = tf.io.read_file(mask)
    mask = tf.io.decode_png(mask)
    # Convert mask to grayscale for it contains one channel
    mask = tf.image.rgb_to_grayscale(mask)
    mask = tf.image.resize(mask, OUTPUT_SIZE)
    mask = tf.image.convert_image_dtype(mask, tf.float32)

    masks = []
    
    # For each class (CLASSES) creates a mask, where 1.0 indicates the presence of the class, and 0.0 - its absence
    for i in range(CLASSES):
        masks.append(tf.where(tf.equal(mask, float(i)), 1.0, 0.0))

    # Combine all class masks into one tensor.
    masks = tf.stack(masks, axis=2)
    # Change the form of the combined mask tensor to match the output format
    masks = tf.reshape(masks, OUTPUT_SIZE+(CLASSES,))

    return image, masks

### Augumintate function

In [50]:
def augmentate_images(image, masks):
    random_crop = tf.random.uniform((), 0.3, 1)
    image = tf.image.central_crop(image, random_crop)
    masks = tf.image.central_crop(masks, random_crop)

    random_flip = tf.random.uniform((), 0, 1)
    if random_flip >=0.5:
        image = tf.image.flip_left_right(image)
        masks = tf.image.flip_left_right(masks)

    image = tf. image.resize(image, SAMPLE_SIZE)
    masks = tf. image.resize(masks, SAMPLE_SIZE)

    return image, masks

### Load data from drive

In [51]:

images = sorted(glob.glob(images_path+'*.jpg'))
masks = sorted(glob.glob(masks_path+'*.png'))

#Formation of data sets from images and masks
images_dataset = tf.data.Dataset.from_tensor_slices(images)
masks_dataset = tf.data.Dataset.from_tensor_slices(masks)

# Union datasets for paralel processing
dataset = tf.data.Dataset.zip((images_dataset, masks_dataset))

# Load images to memory
dataset = dataset.map(image_load, num_parallel_calls = tf.data.AUTOTUNE)

# Increasing the volume of data by copying
dataset = dataset.repeat(50)

# Use augumentate function for all images
dataset = dataset.map(augmentate_images, num_parallel_calls = tf.data.AUTOTUNE)

### Division of sets into training and test sets

In [53]:
train_dataset = dataset.take(2000).cache()
test_dataset = dataset.skip(2000).take(100).cache()

# Set size of packadge
train_dataset = train_dataset.batch(16)
test_dataset = test_dataset.batch(16)